In [625]:
import pandas as pd
import datacompy

In [626]:
import pyodbc
conn = pyodbc.connect('DSN=SysproCompanyB;Trusted_Connection=yes;')
cursor = conn.cursor()

### Stocking Points / Bom Structure

In [580]:
syspro_bom_structure = '''

/* Standard Routing */

WITH GetBomHierarchy AS
(
    SELECT
                  inv.StockCode
                , bs.ParentPart
                , bs.Component
                , 1 AS LevelNumber
    FROM
                SysproCompanyB.dbo.BomStructure bs
    INNER JOIN
                SysproCompanyB.dbo.InvMaster inv
    ON
                bs.ParentPart = inv.StockCode
    WHERE
                bs.Route = 0
    AND
                LEFT(inv.StockCode, 1) <> 5

    UNION ALL

    SELECT
                  gbh.StockCode
                , bs.ParentPart
                , bs.Component
                , gbh.LevelNumber + 1 AS LevelNumber
    FROM
                SysproCompanyB.dbo.BomStructure bs
    INNER JOIN
                GetBomHierarchy gbh
    ON
                bs.ParentPart = gbh.Component
)
    SELECT DISTINCT
                      StockCode AS ParentPart
                    , ParentPart AS Component
                    --, LevelNumber
    FROM
                    GetBomHierarchy lf
'''

In [581]:
syspro_p_bom = pd.read_sql_query(syspro_bom_structure, conn)
syspro_p_bom = syspro_p_bom.sort_values(by='ParentPart')

In [582]:
syspro_p_bom

,ParentPart,Component
2891,10010064,10010064
3994,10010064,50650862
2307,10010064,50110110
3566,10010065,50840338
1906,10010065,10010065
...,...,...
4309,13010000,13010000
4438,13010001,13010001
3447,13010002,13010002
1265,13010003,13010003


In [583]:
quintiq_bom_structure = '''
WITH GetQuintiqBomRoutes AS
(
    SELECT
                  prsm.InputProductId
                , ip.ProductKey AS InputProductKey
                , prsm.ProductId
                , p.ProductKey
                , prsm.RoutingId
                , prsm.StockingPointId AS InputStockingPointId
                , p.StockingPointId AS ProductStockPointId
    FROM    
                QuintiqIDB.dbo.QRG_ProductionRoutingStepInputMaterial prsm
    LEFT JOIN 
                QuintiqIDB.dbo.QRG_ProductionRouting pr
    ON      
                prsm.RoutingId = pr.RoutingId
    LEFT JOIN  
                QuintiqIDB.dbo.QRG_Product p
    ON
                prsm.ProductId = p.ProductId
    LEFT JOIN
                QuintiqIDB.dbo.QRG_Product ip
    ON
                prsm.InputProductId = ip.ProductId
    WHERE   
                prsm.StockingPointId <> 'CON'
    AND 
                pr.Preferred = '1'
)
, GetBomRouteHierarchy AS
(
    SELECT
                  rt.ProductKey AS FinalProductKey
                , rt.InputProductId
                , rt.InputProductKey
                --, rt.ProductId
                , rt.ProductKey
                --, rt.RoutingId
                --, rt.InputStockingPointId
                --, rt.ProductStockPointId
                , 1 AS LevelNumber
    FROM
                GetQuintiqBomRoutes rt
    WHERE
                ProductStockPointId = 'FG'

    UNION ALL

    SELECT
                  rt.FinalProductKey
                , lv.InputProductId
                , lv.InputProductKey
                --, lv.ProductId
                , lv.ProductKey
                --, lv.RoutingId
                --, lv.InputStockingPointId
                --, lv.ProductStockPointId
                , rt.LevelNumber + 1 AS LevelNumber
    FROM
                GetQuintiqBomRoutes lv
    INNER JOIN
                GetBomRouteHierarchy rt
    ON
                rt.InputProductId = lv.ProductId
            
)
SELECT DISTINCT
              --FinalProductKey
            --, InputProductKey
            --, ProductKey
            --, 
            SUBSTRING(FinalProductKey, 4, 8) AS ParentPart
            , ISNULL(invp.StockCode, SUBSTRING(FinalProductKey, 4, 8)) AS Component
            --, LevelNumber
FROM 
            GetBomRouteHierarchy prt
LEFT JOIN
            SysproCompanyB.dbo.[InvMaster+] invp
ON
            prt.ProductKey = invp.QuintiqProductKey;
'''

In [584]:
quintiq_p_bom = pd.read_sql_query(quintiq_bom_structure, conn)
quintiq_p_bom = quintiq_p_bom.sort_values(by='ParentPart')

In [585]:
quintiq_p_bom = quintiq_p_bom[['ParentPart', 'Component']]
quintiq_p_bom

,ParentPart,Component
2325,10010064,50650862
1327,10010064,50110110
1658,10010064,10010064
1085,10010065,10010065
2068,10010065,50840338
...,...,...
890,21000426,50650180
968,21000426,21000426
760,21000427,21000427
1140,21000427,50670467


In [586]:
compare = datacompy.Compare(
    df1 = syspro_p_bom,
    df2 = quintiq_p_bom,
    join_columns=['ParentPart'],  #You can also specify a list of columns
    abs_tol=0, #Optional, defaults to 0
    rel_tol=0, #Optional, defaults to 0
    df1_name ='syspro', #Optional, defaults to 'df1'
    df2_name ='quintiq' #Optional, defaults to 'df2'
    )
compare.matches(ignore_extra_columns=False)
# False

# This method prints out a human-readable report summarizing and sampling differences
#print(compare.report(sample_count=12741))

#csvFileToWrite=r'c:\Users\mwelch\data_comparison.csv'

#with open(csvFileToWrite,mode='r+',encoding='utf-8') as report_file:
        #report_file.write(compare.report(sample_count=14544))
syspro_not_quintiq = compare.df1_unq_rows
quintiq_not_syspro = compare.df2_unq_rows
diff = compare.sample_mismatch('component', sample_count=12741, for_display=True)

#### Components/Stocking Points in Syspro and not in Quintiq

In [587]:
syspro_not_quintiq

,parentpart,component
24,10010073,10010073
25,10010073,50690132
26,10010073,50320128
27,10010074,50690128
28,10010074,50320088
...,...,...
4513,12550041,12550041
4516,12550044,12550044
4517,12550045,12550045
4520,12550048,12550048


#### Components/Stocking Points in Quintiq and not in Syspro

In [588]:
quintiq_not_syspro

,parentpart,component
4535,13010005,13010005
4536,13010006,13010006
4537,13010007,13010007
4538,13010008,13010008
4539,21000420,50670467
4540,21000420,50200199
4541,21000420,21000420
4542,21000421,52300064
4543,21000421,21000421
4544,21000421,52310026


#### Matched Parent with Different Component

In [589]:
diff

,parentpart,component (syspro),component (quintiq)
325,10100124,10100124,50460171
1779,10300051,50340162,10300051
2730,10741008,50170226,10741008
3686,10741341,10741341,52290094
2360,10740461,10740461,50640302
...,...,...,...
3264,10741197,50050209,50650180
202,10070160,10070160,50110162
3803,10741456,10741456,50650786
2243,10740318,10740318,50690363


## Production Routing Steps & Bom Operations

### SysproCompanyB Preferred Routes Query (Code)

In [870]:
# syspro companyb db query
syspro_sql_query = '''

/* Standard Routing */

WITH GetBomHierarchy AS
(
    SELECT
                  inv.StockCode
                , bs.ParentPart
                , bs.Component
                , 1 AS LevelNumber
    FROM
                SysproCompanyB.dbo.BomStructure bs
    INNER JOIN
                SysproCompanyB.dbo.InvMaster inv
    ON
                bs.ParentPart = inv.StockCode
    WHERE
                bs.Route = 0
    AND
                LEFT(inv.StockCode, 1) <> 5

    UNION ALL

    SELECT
                  gbh.StockCode
                , bs.ParentPart
                , bs.Component
                , gbh.LevelNumber + 1 AS LevelNumber
    FROM
                SysproCompanyB.dbo.BomStructure bs
    INNER JOIN
                GetBomHierarchy gbh
    ON
                bs.ParentPart = gbh.Component
)
, GroupInterimProducts AS
(
    SELECT DISTINCT
                      StockCode
                    , LevelNumber
                    , ParentPart
    FROM
                    GetBomHierarchy lf
)
SELECT
              gbh.StockCode
            --, gbh.LevelNumber
            , gbh.ParentPart AS InterimStockCode
            --, CAST(bop.Operation AS INT) Operation
            , bop.WorkCentre
            , ISNULL(NULLIF(bop.ToolSet, ''), 'None') AS ToolSet
            , bop.IQuantity
            , CASE 
                    WHEN bwc.TimeUom = 'hrs' THEN (bop.ITimeTaken * 60)
                    ELSE bop.ITimeTaken
              END 
                    ProductionTimeMins
            --, bwc.TimeUom
FROM 
            GroupInterimProducts gbh
LEFT JOIN
            SysproCompanyB.dbo.BomOperations bop
ON
            gbh.ParentPart = bop.StockCode
LEFT JOIN
            SysproCompanyB.dbo.BomWorkCentre bwc
ON
            bop.WorkCentre = bwc.WorkCentre
LEFT JOIN
            SysproCompanyB.dbo.zWorkCentreCost wcc
ON
            bwc.WorkCentre = wcc.WorkCentre
LEFT JOIN
            SysproCompanyB.dbo.[InvMaster+] invp
ON
            gbh.StockCode = invp.StockCode
LEFT JOIN
            SysproCompanyB.dbo.InvMaster inv
ON
            gbh.StockCode = inv.StockCode
WHERE
            bop.WorkCentre NOT IN ('PACK')
ORDER BY
            gbh.StockCode ASC, gbh.LevelNumber DESC, bop.Operation ASC
'''

### Extract & Tag Data Frame with Data Source

In [871]:
# extract syspro routes from sysprocompanyb
syspro_p_routes = pd.read_sql_query(syspro_sql_query, conn)
syspro_p_routes['source'] = 'SysproCompanyB'
syspro_p_routes

,StockCode,InterimStockCode,WorkCentre,ToolSet,IQuantity,ProductionTimeMins,source
0,10010064,50650862,Scalper1,None,7.260,20.00000,SysproCompanyB
1,10010064,50650862,Preheat1,None,7.260,19.99998,SysproCompanyB
2,10010064,50650862,Hotline1,None,7.260,15.00000,SysproCompanyB
3,10010064,50110110,AnnLC1,None,7.139,510.00000,SysproCompanyB
4,10010064,50110110,TSM1,1_S,7.139,10.11000,SysproCompanyB
...,...,...,...,...,...,...,...
12759,13010004,13010004,Preheat1,None,6.900,19.99998,SysproCompanyB
12760,13010004,13010004,Hotline1,None,6.900,12.91000,SysproCompanyB
12761,13010004,13010004,AnnLC2,None,6.900,720.00000,SysproCompanyB
12762,13010004,13010004,TSM2,4_F,6.900,7.82000,SysproCompanyB


### Quintiq IDB Preferred Routes Query (Code)

In [872]:
# quintiq integration db query
quintiq_sql_query = '''

/* Standard Routing */

WITH GetQuintiqBomRoutes AS
(
    SELECT
                  prsm.InputProductId
                , prsm.ProductId
                , prsm.RoutingId
                , prsm.StockingPointId AS InputStockingPointId
                , p.StockingPointId AS ProductStockPointId
    FROM    
                QuintiqIDB.dbo.QRG_ProductionRoutingStepInputMaterial prsm
    LEFT JOIN 
                QuintiqIDB.dbo.QRG_ProductionRouting pr
    ON      
                prsm.RoutingId = pr.RoutingId
    LEFT JOIN  
                QuintiqIDB.dbo.QRG_Product p
    ON
                prsm.ProductId = p.ProductId
    LEFT JOIN
                QuintiqIDB.dbo.QRG_Product ip
    ON
                prsm.InputProductId = ip.ProductId
    WHERE   
                prsm.StockingPointId <> 'CON'
    AND 
                pr.Preferred = '1'
)
, GetBomRouteHierarchy AS
(
    SELECT
                  rt.ProductId AS FinalProductId
                , rt.InputProductId
                , rt.ProductId
                , rt.RoutingId
                , rt.InputStockingPointId
                , rt.ProductStockPointId
                , 1 AS LevelNumber
    FROM
                GetQuintiqBomRoutes rt
    WHERE
                ProductStockPointId = 'FG'

    UNION ALL

    SELECT
                  rt.FinalProductId
                , lv.InputProductId
                , lv.ProductId
                , lv.RoutingId
                , lv.InputStockingPointId
                , lv.ProductStockPointId
                , rt.LevelNumber + 1 AS LevelNumber
    FROM
                GetQuintiqBomRoutes lv
    INNER JOIN
                GetBomRouteHierarchy rt
    ON
                rt.InputProductId = lv.ProductId
            
)
SELECT  
              fp.Stockcode
            , ISNULL(NULLIF(invp.StockCode, ''), fp.Stockcode) AS InterimStockCode
            , prs.ResourceGroup AS WorkCentre
            , ISNULL(NULLIF(prs.PassCode, ''), 'None') AS ToolSet
            , ISNULL((pp.PieceWeight / 1000), (fp.PieceWeight / 1000)) AS IQuantity
            , ISNULL(prs.ProductionTime,0) * 24 * 60 ProductionTimeMins
FROM    
            QuintiqIDB.dbo.QRG_ProductionRouting pr
INNER JOIN 
            QuintiqIDB.dbo.QRG_ProductionRoutingStep prs
ON      
            pr.RoutingId = prs.RoutingId
INNER JOIN 
            QuintiqIDB.dbo.QRG_ProductionRoutingStepInputMaterial prsm
ON      
            pr.RoutingId = prsm.RoutingId
AND 
            prsm.StockingPointId <> 'CON'
INNER JOIN 
            GetBomRouteHierarchy ot
ON      
            prsm.RoutingId = ot.RoutingId
INNER JOIN 
            QuintiqIDB.dbo.QRG_Product p
ON      
            ot.InputProductId = p.ProductId
LEFT JOIN
            QuintiqIDB.dbo.QRG_Product fp
ON
            ot.FinalProductId = fp.ProductId
LEFT JOIN
            QuintiqIDB.dbo.QRG_Product pp
ON
            ot.ProductId = pp.ProductId
LEFT JOIN
            SysproCompanyB.dbo.[InvMaster+] invp
ON
            pp.ProductKey = invp.QuintiqProductKey
WHERE
            prs.ResourceGroup NOT IN ('Transport1', 'Test1')
ORDER BY 
              fp.Stockcode
            , ot.LevelNumber DESC
            , prs.SequenceNr ASC
'''

### Extract & Tag Data Frame with Data Source

In [873]:
# extract quintiq routes from quintiq integration db
quintiq_p_routes = pd.read_sql_query(quintiq_sql_query, conn)
quintiq_p_routes['source'] = 'QuintiqIDB'
quintiq_p_routes

,Stockcode,InterimStockCode,WorkCentre,ToolSet,IQuantity,ProductionTimeMins,source
0,10010064,50650862,Scalper1,None,7.260,18.00000,QuintiqIDB
1,10010064,50650862,Preheat1,None,7.260,19.99998,QuintiqIDB
2,10010064,50650862,Hotline1,None,7.260,19.17186,QuintiqIDB
3,10010064,50110110,AnnLC1,None,7.139,510.00000,QuintiqIDB
4,10010064,50110110,TSM1,1_S,7.139,9.73704,QuintiqIDB
...,...,...,...,...,...,...,...
7785,21000427,50670467,Hotline1,None,5.040,17.31990,QuintiqIDB
7786,21000427,50530199,AnnLC2,None,4.956,514.99998,QuintiqIDB
7787,21000427,50530199,TSM2,1_S,4.956,12.98502,QuintiqIDB
7788,21000427,21000427,TSM2,4_F,4.191,16.42026,QuintiqIDB


### DataComPY Library

In [874]:
# data compy
compare = datacompy.Compare(
    df1 = syspro_p_routes,
    df2 = quintiq_p_routes,
    join_columns=['StockCode', 'InterimStockCode', 'WorkCentre', 'ToolSet'],  #You can also specify a list of columns
    abs_tol=0, #Optional, defaults to 0
    rel_tol=0, #Optional, defaults to 0
    df1_name ='syspro', #Optional, defaults to 'df1'
    df2_name ='quintiq' #Optional, defaults to 'df2'
    )
compare.matches(ignore_extra_columns=False)
# False

# This method prints out a human-readable report summarizing and sampling differences
print(compare.report(sample_count=12741))

#csvFileToWrite=r'c:\Users\mwelch\data_comparison.csv'

#with open(csvFileToWrite,mode='r+',encoding='utf-8') as report_file:
        #report_file.write(compare.report(sample_count=14544))

time_diff = compare.sample_mismatch('productiontimemins', sample_count=12764, for_display=True)
quantity_diff = compare.sample_mismatch('iquantity', sample_count=12764, for_display=True)

syspro_not_quintiq = compare.df1_unq_rows
quintiq_not_syspro = compare.df2_unq_rows

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0    syspro        7  12764
1   quintiq        7   7790

Column Summary
--------------

Number of columns in common: 7
Number of columns in syspro but not in quintiq: 0
Number of columns in quintiq but not in syspro: 0

Row Summary
-----------

Matched on: stockcode, interimstockcode, workcentre, toolset
Any duplicates on match values: Yes
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 7,380
Number of rows in syspro but not in quintiq: 5,384
Number of rows in quintiq but not in syspro: 410

Number of rows with some compared columns unequal: 7,380
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 3
Number of columns compared with all values equal: 4
Total number of values which compare unequal: 13,869

Columns with Unequal Values or Types
------------------------------

### Tag Data Frames from 'sample_mismatch' Method

In [875]:
#calculate time variance for matching records across data sources
time_diff['variance (mins)'] = time_diff['productiontimemins (syspro)'] - time_diff['productiontimemins (quintiq)']
time_diff

,stockcode,interimstockcode,workcentre,toolset,productiontimemins (syspro),productiontimemins (quintiq),variance (mins)
5229,10210755,10210755,Hotline1,None,13.79000,13.79304,-0.00304
591,10070163,10070163,BWG2,None,37.66000,39.69390,-2.03390
8481,10741084,50690383,Scalper1,None,19.99998,18.00000,1.99998
10874,10780046,50340234,TSM1,1_S,6.74000,8.34990,-1.60990
5269,10210759,10210759,Hotline1,None,13.31000,13.31148,-0.00148
...,...,...,...,...,...,...,...
10226,10741310,50690380,Hotline1,None,15.00000,15.75024,-0.75024
11506,11150017,50671054,Hotline1,None,15.00000,22.29744,-7.29744
11036,10780096,50680019,Scalper1,None,20.00000,18.00000,2.00000
7112,10740480,10740480,BWG2,None,20.88000,25.37592,-4.49592


In [876]:
# calculate quantity variance for matching records across data sources
quantity_diff['variance'] = quantity_diff['iquantity (syspro)'] - quantity_diff['iquantity (quintiq)']
quantity_diff

,stockcode,interimstockcode,workcentre,toolset,iquantity (syspro),iquantity (quintiq),variance
3762,10210561,50510075,TSM2,2_I,8.083,8.220,-0.137
5213,10210753,50540290,TSM2,2_I,9.204,9.360,-0.156
4691,10210695,10210695,TSM2,1_S,8.070,7.010,1.060
4860,10210714,10210714,TSM2,4_F,6.460,5.600,0.860
4630,10210689,10210689,Hotline1,None,6.460,5.600,0.860
...,...,...,...,...,...,...,...
549,10070158,50480160,TSM2,1_H,7.729,7.860,-0.131
4628,10210689,10210689,Scalper1,None,6.460,5.600,0.860
4366,10210654,10210654,TSM2,3_I,4.846,4.200,0.646
3841,10210572,50490207,TSM2,1_H,6.313,6.955,-0.642


### Tag Data Frames from 'df_unq_rows' Method

In [877]:
# syspro_not_quintiq
# add boolean column set to true
syspro_not_quintiq['not_in_quintiq'] = 1
syspro_not_quintiq

,stockcode,interimstockcode,workcentre,toolset,iquantity,productiontimemins,source,not_in_quintiq
13,10010065,10010065,TSM1,4_F,6.05,12.13000,SysproCompanyB,1
53,10010070,10010070,TSM1,4_F,6.05,13.71000,SysproCompanyB,1
64,10010073,50690132,Scalper1,None,9.06,20.00000,SysproCompanyB,1
65,10010073,50690132,Preheat1,None,9.06,19.99998,SysproCompanyB,1
66,10010073,50690132,Hotline1,None,9.06,15.00000,SysproCompanyB,1
...,...,...,...,...,...,...,...,...
12749,13010002,13010002,TSM2,4_F,4.60,8.22000,SysproCompanyB,1
12750,13010002,13010002,ES1,None,4.60,24.20000,SysproCompanyB,1
12756,13010003,13010003,TSM2,4_F,4.60,8.22000,SysproCompanyB,1
12757,13010003,13010003,ES1,None,4.60,24.20000,SysproCompanyB,1


In [878]:
# quintiq_not_syspro
# add boolean column set to true
quintiq_not_syspro['not_in_syspro'] = 1
quintiq_not_syspro

,stockcode,interimstockcode,workcentre,toolset,iquantity,productiontimemins,source,not_in_syspro
12764,10010065,10010065,TSM2,4_F,6.050,16.22664,QuintiqIDB,1
12765,10010070,10010070,TSM2,4_F,6.050,17.82006,QuintiqIDB,1
12766,10010082,10010082,TSM2,4_F,6.180,16.22664,QuintiqIDB,1
12767,10010089,10010089,TSM2,4_F,6.330,16.22664,QuintiqIDB,1
12768,10010095,10010095,TSM2,4_F,6.050,16.42026,QuintiqIDB,1
...,...,...,...,...,...,...,...,...
13169,21000427,50670467,Hotline1,None,5.040,17.31990,QuintiqIDB,1
13170,21000427,50530199,AnnLC2,None,4.956,514.99998,QuintiqIDB,1
13171,21000427,50530199,TSM2,1_S,4.956,12.98502,QuintiqIDB,1
13172,21000427,21000427,TSM2,4_F,4.191,16.42026,QuintiqIDB,1


### Syspro Company B

#### Data Leakage

In [879]:
# test existence / leakage
syspro_p_routes = syspro_p_routes.merge(syspro_not_quintiq[['stockcode', 'interimstockcode', 'workcentre', 'toolset', 
                                                            'not_in_quintiq']], how='left', left_on=['stockcode', 
                                                            'interimstockcode', 'workcentre', 'toolset'], right_on=['stockcode', 
                                                            'interimstockcode', 'workcentre', 'toolset'])
syspro_p_routes['not_in_quintiq'] = syspro_p_routes['not_in_quintiq'].fillna(value=0)

#### Time Variance

In [880]:
# production minutes
syspro_p_routes = syspro_p_routes.merge(time_diff[['stockcode', 'interimstockcode', 'workcentre', 'toolset', 
                                                   'productiontimemins (quintiq)']], how='left', left_on=['stockcode', 
                                                   'interimstockcode', 'workcentre', 'toolset'], right_on=['stockcode', 
                                                   'interimstockcode', 'workcentre', 'toolset'])

In [881]:
# calculate time variance
syspro_p_routes['time_variance'] = syspro_p_routes['productiontimemins'] - syspro_p_routes['productiontimemins (quintiq)']
syspro_p_routes

,stockcode,interimstockcode,workcentre,toolset,iquantity,productiontimemins,source,not_in_quintiq,productiontimemins (quintiq),time_variance
0,10010064,50650862,Scalper1,None,7.260,20.00000,SysproCompanyB,0.0,18.00000,2.00000
1,10010064,50650862,Preheat1,None,7.260,19.99998,SysproCompanyB,0.0,NaN,NaN
2,10010064,50650862,Hotline1,None,7.260,15.00000,SysproCompanyB,0.0,19.17186,-4.17186
3,10010064,50110110,AnnLC1,None,7.139,510.00000,SysproCompanyB,0.0,NaN,NaN
4,10010064,50110110,TSM1,1_S,7.139,10.11000,SysproCompanyB,0.0,9.73704,0.37296
...,...,...,...,...,...,...,...,...,...,...
12791,13010004,13010004,Preheat1,None,6.900,19.99998,SysproCompanyB,0.0,NaN,NaN
12792,13010004,13010004,Hotline1,None,6.900,12.91000,SysproCompanyB,0.0,12.91458,-0.00458
12793,13010004,13010004,AnnLC2,None,6.900,720.00000,SysproCompanyB,0.0,NaN,NaN
12794,13010004,13010004,TSM2,4_F,6.900,7.82000,SysproCompanyB,0.0,7.92876,-0.10876


#### Quantity/Weight Variance

In [882]:
# # quantity/weight variance
# syspro_p_routes = syspro_p_routes.merge(quantity_diff[['stockcode', 'interimstockcode', 'workcentre', 'toolset', 
#                                                    'iquantity (quintiq)']], how='left', left_on=['stockcode', 
#                                                    'interimstockcode', 'workcentre', 'toolset'], right_on=['stockcode', 
#                                                    'interimstockcode', 'workcentre', 'toolset'])

In [883]:
# # calculate time variance
# syspro_p_routes['qty_variance'] = syspro_p_routes['iquantity'] - syspro_p_routes['iquantity (quintiq)']
# syspro_p_routes

### Quintiq IDB Preferred Routes

#### Data Leakage

In [884]:
# test existence / leakage
quintiq_p_routes = quintiq_p_routes.merge(quintiq_not_syspro[['stockcode', 'interimstockcode', 'workcentre', 'toolset', 
                                                              'not_in_syspro']], how='left', left_on=['stockcode', 
                                                              'interimstockcode', 'workcentre', 'toolset'], right_on=['stockcode', 
                                                              'interimstockcode', 'workcentre', 'toolset'])
quintiq_p_routes['not_in_syspro'] = quintiq_p_routes['not_in_syspro'].fillna(value=0)
quintiq_p_routes

,stockcode,interimstockcode,workcentre,toolset,iquantity,productiontimemins,source,not_in_syspro
0,10010064,50650862,Scalper1,None,7.260,18.00000,QuintiqIDB,0.0
1,10010064,50650862,Preheat1,None,7.260,19.99998,QuintiqIDB,0.0
2,10010064,50650862,Hotline1,None,7.260,19.17186,QuintiqIDB,0.0
3,10010064,50110110,AnnLC1,None,7.139,510.00000,QuintiqIDB,0.0
4,10010064,50110110,TSM1,1_S,7.139,9.73704,QuintiqIDB,0.0
...,...,...,...,...,...,...,...,...
7785,21000427,50670467,Hotline1,None,5.040,17.31990,QuintiqIDB,1.0
7786,21000427,50530199,AnnLC2,None,4.956,514.99998,QuintiqIDB,1.0
7787,21000427,50530199,TSM2,1_S,4.956,12.98502,QuintiqIDB,1.0
7788,21000427,21000427,TSM2,4_F,4.191,16.42026,QuintiqIDB,1.0


#### Time Variance

In [885]:
# production minutes
quintiq_p_routes = quintiq_p_routes.merge(time_diff[['stockcode', 'interimstockcode', 'workcentre', 'toolset', 
                                                              'productiontimemins (syspro)']], how='left', left_on=['stockcode', 
                                                              'interimstockcode', 'workcentre', 'toolset'], right_on=['stockcode', 
                                                              'interimstockcode', 'workcentre', 'toolset'])

In [886]:
# calculate time variance
quintiq_p_routes['time_variance'] = quintiq_p_routes['productiontimemins'] - quintiq_p_routes['productiontimemins (syspro)']
quintiq_p_routes

,stockcode,interimstockcode,workcentre,toolset,iquantity,productiontimemins,source,not_in_syspro,productiontimemins (syspro),time_variance
0,10010064,50650862,Scalper1,None,7.260,18.00000,QuintiqIDB,0.0,20.00,-2.00000
1,10010064,50650862,Preheat1,None,7.260,19.99998,QuintiqIDB,0.0,NaN,NaN
2,10010064,50650862,Hotline1,None,7.260,19.17186,QuintiqIDB,0.0,15.00,4.17186
3,10010064,50110110,AnnLC1,None,7.139,510.00000,QuintiqIDB,0.0,NaN,NaN
4,10010064,50110110,TSM1,1_S,7.139,9.73704,QuintiqIDB,0.0,10.11,-0.37296
...,...,...,...,...,...,...,...,...,...,...
7799,21000427,50670467,Hotline1,None,5.040,17.31990,QuintiqIDB,1.0,NaN,NaN
7800,21000427,50530199,AnnLC2,None,4.956,514.99998,QuintiqIDB,1.0,NaN,NaN
7801,21000427,50530199,TSM2,1_S,4.956,12.98502,QuintiqIDB,1.0,NaN,NaN
7802,21000427,21000427,TSM2,4_F,4.191,16.42026,QuintiqIDB,1.0,NaN,NaN


#### Quantity/Weight Variance

In [887]:
# # qty/tonnes variance
# quintiq_p_routes = quintiq_p_routes.merge(quantity_diff[['stockcode', 'interimstockcode', 'workcentre', 'toolset', 
#                                                               'iquantity (syspro)']], how='left', left_on=['stockcode', 
#                                                               'interimstockcode', 'workcentre', 'toolset'], right_on=['stockcode', 
#                                                               'interimstockcode', 'workcentre', 'toolset'])

In [888]:
# # calculate qty variance
# quintiq_p_routes['qty_variance'] = quintiq_p_routes['iquantity'] - quintiq_p_routes['iquantity (syspro)']
# quintiq_p_routes

### Combine Final Data Set

In [902]:
# combine, append, union the two data frames
combined_rts = pd.concat([syspro_p_routes, quintiq_p_routes])

In [903]:
combined_rts = combined_rts[['stockcode', 'interimstockcode', 'workcentre', 'toolset', 'source', 'not_in_quintiq', 'not_in_syspro', 
              #'iquantity', 'iquantity (syspro)', 'iquantity (quintiq)', 'qty_variance', 
                             'productiontimemins', 
              'productiontimemins (syspro)', 'productiontimemins (quintiq)', 'time_variance']]

combined_rts['not_in_syspro'] = combined_rts['not_in_syspro'].fillna(value='N/A')
combined_rts['not_in_quintiq'] = combined_rts['not_in_quintiq'].fillna(value='N/A')
combined_rts['productiontimemins (syspro)'] = combined_rts['productiontimemins (syspro)'].fillna(value='N/A')
combined_rts['productiontimemins (quintiq)'] = combined_rts['productiontimemins (quintiq)'].fillna(value='N/A')
combined_rts['time_variance'] = combined_rts['time_variance'].fillna(value=0)

combined_rts

,stockcode,interimstockcode,workcentre,toolset,source,not_in_quintiq,not_in_syspro,productiontimemins,productiontimemins (syspro),productiontimemins (quintiq),time_variance
0,10010064,50650862,Scalper1,None,SysproCompanyB,0,N/A,20.00000,N/A,18,2.00000
1,10010064,50650862,Preheat1,None,SysproCompanyB,0,N/A,19.99998,N/A,N/A,0.00000
2,10010064,50650862,Hotline1,None,SysproCompanyB,0,N/A,15.00000,N/A,19.1719,-4.17186
3,10010064,50110110,AnnLC1,None,SysproCompanyB,0,N/A,510.00000,N/A,N/A,0.00000
4,10010064,50110110,TSM1,1_S,SysproCompanyB,0,N/A,10.11000,N/A,9.73704,0.37296
...,...,...,...,...,...,...,...,...,...,...,...
7799,21000427,50670467,Hotline1,None,QuintiqIDB,N/A,1,17.31990,N/A,N/A,0.00000
7800,21000427,50530199,AnnLC2,None,QuintiqIDB,N/A,1,514.99998,N/A,N/A,0.00000
7801,21000427,50530199,TSM2,1_S,QuintiqIDB,N/A,1,12.98502,N/A,N/A,0.00000
7802,21000427,21000427,TSM2,4_F,QuintiqIDB,N/A,1,16.42026,N/A,N/A,0.00000


In [900]:
combined_rts.to_csv('data_comparison.csv')